# Final Project Notebook

## Project Overview

who are the stakeholders?

Covid-19 has pushed many business online that may have not had a strong online presence. Pivoting

write about e-commerce impact from recommendations.

Book-recommender system is a proof of concept to encourage the new busnisses that have launched online to incorporate a rating and recommender system.



## Data Understanding

In this section you will find...




#### Obtaining the Data

Explain web-scrapping process for reproducibility.

Where to find the functions, how to do it. why did it?

#### Data Properties

How relates to the real world problem.

explain what features the dataset has, which ones will be used for this phase of the project. 

Write about how want to have a good recommendation because want customers to trust the recommendation system so that they continue to use it in the future. 

## Data Preparation and Exploratory Data Analysis

In this section you will find...

#### Import Libraries

The following cell has the libraries needed to perform the operations in this notebook

#### Import Dataset

As mentioned in the previous section, this dataset is a combination of the multiple tables that were exported during the web scrapping process. 

#### Inspect Dataset

#### Data Preprocessing

Because my scrapper took reviews based on books found on a specific genre's page, there were duplicates between the book ratings because they may have been classified under multiple genres--such as one book falling into fiction, historical fiction, and history genres.

During the data cleaning process I:
- combined the multiple genre book and reviews table to make one combined data frame
- took out the duplicate data left by the scrapping
- removed extraneous text
- removed text in mixed data type columns, such as the word "pages" from the pages columns so that column could be an integer data type


EXPLAIN REASONING BEHIND EACH DECISION.

## Modeling

During the modeling process...

EXPLAIN pyspark ALS


## Evaluation

Cross validation is....

in the context of the project it means...